# **Machine Learning Model For Analyze Stress Level In Students**

**An AI-powered stress classification tool that analyzes student lifestyle, emotional, physical, academic, and social factors to predict stress types and provide useful recommendations**

**Dataset:https://www.kaggle.com/datasets/mdsultanulislamovi/student-stress-monitoring-datasets**

**Purpose: Predict stress type (Eustress / Distress / No Stress) based on student data.**

**Usage: This model can be deployed in web/mobile apps for stress monitoring.**

1)Importing Modules

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

2)Load Dataset

In [ ]:
df = pd.read_csv("StressLevelDataset.csv")

3)Encode Categorical Data

In [ ]:
df["stress_level"] = LabelEncoder().fit_transform(df["stress_level"])

4)Features & Target

In [ ]:
X = df.drop("stress_level", axis=1)
y = df["stress_level"]

5)Scale numeric features

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

6)Creating train and test dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

7)Create Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(len(np.unique(y)), activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

8)Training

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    verbose=1
)

9)Plot training history

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

10)Model Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = np.argmax(model.predict(X_test), axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=["Eustress","Distress","No Stress"]))


11)Results Visualization

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Eustress","Distress","No Stress"],
            yticklabels=["Eustress","Distress","No Stress"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

12)Model Saving

In [ ]:
model.save("stress_model.h5")

13)Save as TFite(For Mobile or Web)(Optional)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("stress_model.tflite", "wb").write(tflite_model)